# Google Gen AI Exchange Hackathon


*   **Organization - bigbasket**
*   **Challenge Objective** : Develop an AI solution that revolutionizes promotional content creation.
*   **Team name** : Team_Supramental
*   **Team members** :
    *   Shiv Choudhury
    *   Reetesh Sahoo
    *   Parimita Sahoo
    *   Radhabaran Mohanty

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip3 install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 46.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
%pip install --upgrade --quiet  langchain-google-genai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.1 MB/s eta 0:00:00


### **Authenticate your notebook environment (Colab only)**

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [4]:
import os
import cv2
from google.colab import userdata
import vertexai
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr

In [5]:
API_KEY = userdata.get('GEMINI_API')
os.environ["GOOGLE_API_KEY"] = API_KEY

In [6]:
project_id = 'noted-flux-434905-f8'
LOCATION = "us-central1"

vertexai.init(project=project_id, location=LOCATION)

In [7]:
def create_img(subject_inp, festival_inp, arrangement_inp, lighting_inp,
                  angle_lense_inp, style_inp, color_palette_inp, background_inp,
                  texture_inp, artistic_element_inp, promo_text_inp):

  llm = ChatGoogleGenerativeAI(temperature= 0.8, model="gemini-1.5-pro-latest")

  system_template = """You are an expert sales and collateral maker. /
  Create an effecient and creattive prompt within 70 words to generate /
  high quality image in 200 dpi"""
  prompt1 = PromptTemplate(template=system_template)

  image_desc = """
  1. {subject} is the main focus of the image
  2. {festival}
  3. {arrangement}
  4. {lighting}
  5. {angle_lense}
  6. {style}
  7. {color_palette}
  8. {background}
  9. {texture}
  10. {artistic_element}
  11. {promo_text}

  Add a title to the center of the image that reads, “{promo_text}” in orange
  block letters.
  """

  prompt2 = PromptTemplate(
    input_variables = ['subject',
                       'festival',
                       'arrangement',
                       'lighting',
                       'angle_lense',
                       'style',
                       'color_palette',
                       'background',
                       'texture',
                       'artistic_element',
                       'promo_text'],
    template = image_desc
  )

  composite_prompt = prompt1 + prompt2
  parser = StrOutputParser()

  chain = composite_prompt | llm | parser

  response_1 = chain.invoke({"subject": subject_inp,
                           "festival": festival_inp,
                           "arrangement": arrangement_inp,
                           "lighting": lighting_inp,
                           "angle_lense": angle_lense_inp,
                           "style": style_inp,
                           "color_palette": color_palette_inp,
                           "background": background_inp,
                           "texture": texture_inp,
                           "artistic_element": artistic_element_inp,
                           "promo_text": promo_text_inp
                           })
  generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-001")

  image = generation_model.generate_images(
    prompt=response_1,
    number_of_images=1,
    aspect_ratio="1:1",
    safety_filter_level="block_some",
    person_generation="allow_adult")

  img_out = image[0]._pil_image

  return img_out



In [8]:
img = create_img('grocery', 'diwali', 'well stock store', 'clear winter light',
                           'wide angle', 'photorealistic', 'vibrant', 'lighted',
                            'glossy', 'light', 'exciting offers')

In [9]:
demo = gr.Interface(fn=create_img,
                    inputs=['text', 'text', 'text', 'text', 'text', 'text',
                            'text', 'text', 'text', 'text', 'text'],
                    outputs="image")
                    # outputs=gr.Gallery(label="output image"))
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://38af9a02223e8daaf7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


3. Arrangement: [User input] (e.g., overflowing basket of autumn produce, a table set for a festive dinner, a well-stocked pantry)

4. Lighting: [User input] (e.g., warm golden sunlight streaming through a window, soft spotlights highlighting a bowl of fruit)

5. Angles & Lenses: [User input] (e.g., wide-angle view of a bustling grocery store, a close-up macro shot of a juicy strawberry)

6. Style: [User input] (e.g., realistic depiction of a Thanksgiving turkey, minimalist arrangement of pantry staples, vibrant colors of a festive fruit bowl)

7. Color Palette: (e.g., warm autumn colors, vibrant Diwali colors, cool and calming Ramadan colors)

8. Background: (e.g., rustic wooden table, white marble countertop, blurred grocery store aisle)

9. Texture: (e.g., rough burlap sack, smooth ceramic bowl, glossy magazine cover)

10. Artistic Elements: (e.g., bokeh effect, light flares, painterly brushstrokes)

In [ ]:
response_1

'"A wide-angle, photorealistic image (200 DPI) of a well-stocked grocery store during Diwali, bathed in clear winter light. Vibrant colors, glossy surfaces, and festive lighting create an atmosphere of excitement.  Focus on the abundance and variety of products. Add a title in the center that reads "EXCITING OFFERS" in orange block letters." \n'